In [ ]:
'''pip install yfinance'''
'''pip install nbimporter'''
'''pip install psycopg2'''

In [44]:
#pip install psycopg2

In [45]:
#pip install nbimporter

In [42]:
#pip install yfinance

In [1]:
import yfinance as yf
from datetime import datetime
import pandas as pd
import psycopg2
import configparser
import datetime
import nbimporter
from General import *
from Daily_etl import *
from SQL import *
from Functions import *

In [2]:
#connecting to the database


def connect_to_database_posidon():
    try:
        # Read database credentials from the configuration file
        config = configparser.ConfigParser()
        config.read('configuration.ini')

        host = config['database']['host']
        user = config['database']['user']
        password = config['database']['password']
        database = config['database']['database']

        # Establish a connection to the database
        connection = psycopg2.connect(host=host, user=user, password=password, database=database)

        return connection

    except (Exception, psycopg2.Error) as error:
        print(f"Error: {error}")
        return None

if __name__ == "__main__":
    # Establish a connection to the database
    connection = connect_to_database_posidon()

    if connection:
        print("Connected to the database successfully.")
        # can continue with database operations here

        # Close the connection when done
        connection.close()
    else:
        print("Failed to connect to the database.")


Connected to the database successfully.


In [3]:

# Function to execute SQL queries
#SQL REFERENCE A


def execute_sql_query_poisedon(query, params=None):
    
    # Establish a connection to the database
    connection = connect_to_database_posidon()

    try:
        if connection:
            cursor = connection.cursor()
            
            # Execute the SQL query
            if params:
                cursor.execute(query, params)
            else:
                cursor.execute(query)

            # If it's a SELECT query, fetch the results
            if query.strip().lower().startswith("select"):
                results = cursor.fetchall()
                cursor.close()
                return results

            # For other types of queries, commit the changes
            else:
                connection.commit()
               # print("Query executed successfully.")

    except (Exception, psycopg2.Error) as error:
        print(f"Error: {error}")
        if connection:
            connection.rollback()

    finally:
        if connection:
            connection.close()

In [28]:
##Testing For rows

In [25]:
#This is a sample test and will be used for testing only. 
#It utilises yfinance library. It takes "Ticker" symbol then returns the ticker information. 
#Fetch data for a single stock symbol
#This is used to get financial information on stock market from yahoo finance using ticker as test 

stock = yf.Ticker("0A0B.L")
hist = stock.history(period="6y")  # Fetches one month of historical data

print(hist)
#If you want to see the result just uncomment the print statement.
#This was a test that was run to ensure that the code is able to work on one ticker


                                  Open         High          Low        Close  \
Date                                                                            
2019-05-02 00:00:00+01:00   273.756256   273.756256   273.756256   273.756256   
2019-05-23 00:00:00+01:00   276.249390   276.249390   276.249390   276.249390   
2019-06-17 00:00:00+01:00   277.191681   277.191681   277.191681   277.191681   
2019-06-18 00:00:00+01:00   279.056641   279.056641   279.056641   279.056641   
2019-06-19 00:00:00+01:00   277.682465   277.682465   277.682465   277.682465   
...                                ...          ...          ...          ...   
2024-04-12 00:00:00+01:00  7254.429199  7263.049805  7247.616699  7247.616699   
2024-04-15 00:00:00+01:00  7260.049805  7260.049805  7253.982910  7253.982910   
2024-04-17 00:00:00+01:00  7244.528809  7254.294922  7244.528809  7254.294922   
2024-04-19 00:00:00+01:00  7262.946777  7267.740234  7250.259766  7257.000488   
2024-04-23 00:00:00+01:00  7

In [31]:
#I know need to insert Symbol and description into the poseidon.

In [75]:
'''#path to the text file 
file_path = "lse.txt"

#SQL query to insert data into the table
insert_query = "INSERT INTO \"ticker_and_name\" (\"Symbol\", \"description\") VALUES (%s, %s);"

#connection to the database and creates a cursor
connection = connect_to_database_posidon()
cursor = connection.cursor()

try:
    # Opens the file for reading
    with open(file_path, "r") as file:
        # Reads each line in the file
        for line in file:
            
            # Splits each line into columns based on a delimiter (e.g., tab)
            parts = line.strip().split('\t')  # Assumes tab as the delimiter

            # Check if the line contains the expected number of columns
            if len(parts) == 2:  # Assuming 2 columns (Symbol and Description)
                symbol, description = parts

                # Executes the INSERT query for each row
                cursor.execute(insert_query, (symbol, description))

    # Commits the changes to the database
    connection.commit()
    print("Data inserted successfully.")

except (Exception, psycopg2.Error) as error:
    print(f"Error: {error}")
    if connection:
        connection.rollback()

finally:
    # Closes the cursor and connection
    if cursor:
        cursor.close()
    if connection:
        connection.close()
'''

Data inserted successfully.


In [9]:
#NOW THAT SYMBOLS AND DESCRIPTION HAVE BEEN ADDED TO poseidon THE REST CAN CONTINUE. 

In [ ]:
#INSERT STOCK INFORMATION Into poseidon s_historical_stock_data the bwlow code does that

In [21]:
def fetch_historical_data_posiedon(symbol):
    try:
        stock = yf.Ticker(symbol)
        end_date = datetime.now().strftime('%Y-%m-%d')  # Explicitly calling now() function
        hist = stock.history(start="2005-01-01", end=end_date)
        if hist.empty:
            return pd.DataFrame()
        
        hist.reset_index(inplace=True)
        hist['Symbol'] = symbol

        #the desired column order, 
        #Because I was receiving lots of columns that were not important, therefore I only want the specified columns.
        desired_columns =['Symbol', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits' ]
        return hist[desired_columns]
    except Exception as e:
        print(f"Error fetching data for {symbol}: {str(e)}")
        return pd.DataFrame()

In [26]:
from datetime import datetime

# Path to file containing symbols
file_path = "lse.txt"
symbols = []

# Flag to start adding symbols
start_adding = True

with open(file_path, "r") as file:
    next(file)  # Skip the header
    for line in file:
        symbol = line.split()[0].strip()
        # Skip invalid symbols
        if symbol in ('null', '0.0', ''):
            continue
        if start_adding:
            symbols.append(symbol)


#SQL INSERT statement here
insert_query = """
    INSERT INTO s_historical_stock_data (symbol, date, open, high, low, close, volume, dividends, stock_splits)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Process symbols in batches of 100
batch_size = 100
for i in range(0, len(symbols), batch_size):
    batch_symbols = symbols[i:i + batch_size]
    all_historical_data = pd.DataFrame()

    for symbol in batch_symbols:
        historical_data = fetch_historical_data_posiedon(symbol)
        if not historical_data.empty:
            all_historical_data = pd.concat([all_historical_data, historical_data])

    if not all_historical_data.empty:
        records_to_insert = all_historical_data.to_records(index=False).tolist()

        record_count = 0
        for record in records_to_insert:
            # Handle NaN values and convert them to None
            modified_record = tuple([None if pd.isna(value) else value for value in record])

            execute_sql_query_poisedon(insert_query, modified_record)
            record_count += 1

        print(f"Batch {i//batch_size + 1}: Inserted {record_count} records.")
    else:
        print(f"Batch {i//batch_size + 1}: No data to insert.")
        


02NG.L: Period '1mo' is invalid, must be one of ['1d', '5d']
0DK9.L: No timezone found, symbol may be delisted
0DU3.L: No timezone found, symbol may be delisted
0E12.L: No timezone found, symbol may be delisted


KeyboardInterrupt: 

In [ ]:
#NEXT STEP IS TO POPULATE S_CUSTOMER DATE ON POSIEDON.